# 📊 Hockey Prediction System - Risk Assessment Analysis

**Comprehensive Risk Assessment for Hockey Betting Strategies**

---

## 📋 Analysis Overview

This notebook provides comprehensive risk assessment for hockey betting strategies including:

- **Maximum Drawdown Analysis** - Peak-to-trough analysis with recovery periods
- **Monte Carlo VaR Simulations** - 10,000 scenario risk modeling  
- **Risk-Adjusted Performance** - Sharpe, Sortino, Calmar, Omega ratios
- **Portfolio Correlation Analysis** - Diversification and correlation mapping
- **Recovery Period Analysis** - Drawdown recovery resilience
- **Executive Summary & Reporting** - Strategic recommendations and export

---

## 📊 Key Features

✅ **Statistical Rigor:** 10,000 Monte Carlo simulations with bootstrap resampling  
✅ **Professional Visualizations:** Interactive Plotly charts with HTML export  
✅ **Comprehensive Reporting:** Multi-format export (JSON, CSV, HTML)  
✅ **Executive Insights:** Strategic decision framework with go/no-go assessment  
✅ **Portfolio Optimization:** Correlation-based allocation recommendations  

---

## 🎯 Analysis Results Summary

**This cell will be populated after running the analysis:**

- Portfolio Quality Score: ___/100
- Overall Risk Rating: ___
- Go/No-Go Decision: ___
- Strategic Recommendations: ___ items
- Visualizations Created: ___ charts
- Reports Exported: ___ files

---

*Analysis Framework: Comprehensive Hockey Betting Strategy Risk Assessment v1.0*

---

# 🔧 PART 1: PROJECT SETUP & DATA PIPELINE

**Robust project setup with intelligent data loading and strategy selection**

---

In [ ]:
# ===================================================================
# 📊 RISK ASSESSMENT ANALYSIS - PART 1: PROJECT SETUP & DATA PIPELINE
# ===================================================================
# 
# Tento notebook provádí comprehensive risk assessment analýzu
# top betting strategií s focus na:
# - Maximum drawdown analysis
# - Monte Carlo VaR simulations (10,000 scenarios)
# - Risk-adjusted performance metrics
# - Portfolio correlation analysis
# - Recovery period analysis
#
# Location: notebooks/analysis/risk_assessment.ipynb
# Part 1/4: Project Setup & Data Pipeline
# ===================================================================

import pandas as pd
import numpy as np
import json
import os
import sys
import glob
import warnings
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Any, Optional, Union
import logging
import re

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [ ]:
# ===================================================================
# 📂 SECTION 1: PROJECT PATHS & SETUP
# ===================================================================

print("🔧 Setting up project paths and environment...")
print("=" * 60)

# Get notebook directory and project root
NOTEBOOK_DIR = Path.cwd()
if 'notebooks/analysis' in str(NOTEBOOK_DIR):
    PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
else:
    # Fallback for different execution contexts
    PROJECT_ROOT = Path('../../') if Path('../../requirements.txt').exists() else Path('./')

# Define all critical paths
PATHS = {
    'project_root': PROJECT_ROOT,
    'results_dir': PROJECT_ROOT / 'models' / 'experiments',
    'charts_export_dir': PROJECT_ROOT / 'models' / 'experiments' / 'charts',
    'logs_dir': PROJECT_ROOT / 'logs',
    'src_dir': PROJECT_ROOT / 'src',
    'data_dir': PROJECT_ROOT / 'data',
    'notebook_dir': NOTEBOOK_DIR
}

# Ensure all directories exist
for dir_name, dir_path in PATHS.items():
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✅ Directory exists: {dir_name} -> {dir_path}")

# Add src directory to Python path for imports
sys.path.insert(0, str(PATHS['src_dir']))

print(f"\n🎯 Project root: {PROJECT_ROOT}")
print(f"📊 Results directory: {PATHS['results_dir']}")
print(f"📈 Charts export: {PATHS['charts_export_dir']}")

In [ ]:
# ===================================================================
# 📚 LIBRARY IMPORTS & SETUP
# ===================================================================

print("\n📚 Loading required libraries...")
print("=" * 60)

# Visualization libraries
try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.offline as pyo
    PLOTLY_AVAILABLE = True
    print("✅ Plotly loaded for interactive visualizations")
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️  Plotly not available, falling back to matplotlib")
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.style.use('default')

# Statistical libraries
from scipy import stats
from scipy.stats import norm
import scipy.optimize as optimize

# Try to import custom modules (with fallback)
try:
    from betting.backtesting_engine import BacktestingEngine
    from betting.performance_analyzer import PerformanceAnalyzer
    CUSTOM_MODULES_AVAILABLE = True
    print("✅ Custom betting modules loaded successfully")
except ImportError as e:
    CUSTOM_MODULES_AVAILABLE = False
    print(f"⚠️  Custom betting modules not available: {e}")

# Database import - optional, not critical for analysis
try:
    from utils.database import DatabaseManager
    DATABASE_AVAILABLE = True
    print("✅ Database module loaded")
except ImportError:
    DATABASE_AVAILABLE = False
    print("⚠️  Database module not available (not critical for analysis)")

print("   Continuing with core functionality...")

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(PATHS['logs_dir'] / 'risk_assessment.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("✅ All libraries loaded successfully!")

In [ ]:
# ===================================================================
# 🔍 SECTION 2: DATA FILE DETECTION & LOADING
# ===================================================================

class DataManager:
    """Simplified data management with quality assessment"""
    
    def __init__(self, results_dir: Path):
        self.results_dir = results_dir
        # Mapování vzorů souborů na interní názvy
        self.data_patterns = {
            'top_strategies_bets': 'backtest_top_strategies_bets_*.csv',
            'top_strategies_daily': 'backtest_top_strategies_daily_financial_*.csv', 
            'top_strategies_monthly': 'backtest_top_strategies_monthly_*.csv',
            'strategy_correlations': 'backtest_strategy_correlation_*.csv',
            'top_strategies_summary': 'backtest_top_strategies_summary_*.json',
        }
    
    def extract_timestamp(self, filepath: Path) -> datetime:
        """Extract timestamp from filename"""
        filename = filepath.name
        
        # Try different timestamp patterns
        patterns = [
            (r'(\d{8}_\d{6})', '%Y%m%d_%H%M%S'),
            (r'(\d{14})', '%Y%m%d%H%M%S'),
            (r'(\d{8})', '%Y%m%d'),
        ]
        
        for pattern, date_format in patterns:
            match = re.search(pattern, filename)
            if match:
                try:
                    return datetime.strptime(match.group(1), date_format)
                except ValueError:
                    continue
        
        # Fallback to file modification time
        return datetime.fromtimestamp(filepath.stat().st_mtime)
    
    def find_latest_file(self, pattern: str) -> Optional[Path]:
        """Find the latest file matching pattern"""
        files = list(self.results_dir.glob(pattern))
        if not files:
            return None
        return max(files, key=self.extract_timestamp)
    
    def assess_data_quality(self, df: pd.DataFrame, data_type: str) -> Dict[str, Any]:
        """Simplified data quality assessment"""
        quality_metrics = {
            'total_rows': len(df),
            'total_columns': len(df.columns),
            'missing_values': df.isnull().sum().to_dict(),
            'data_type': data_type
        }
        
        # Calculate quality score
        max_missing_pct = max(df.isnull().sum() / len(df) * 100) if len(df) > 0 else 0
        quality_score = max(0, 100 - max_missing_pct - (10 if len(df) < 100 else 0))
        quality_metrics['quality_score'] = quality_score
        
        return quality_metrics
    
    def load_all_data(self) -> Dict[str, Any]:
        """Load all available data files"""
        print("\n📊 LOADING DATA FILES")
        print("=" * 60)
        
        loaded_data = {}
        quality_reports = {}
        
        for data_type, pattern in self.data_patterns.items():
            print(f"\n📂 Loading {data_type}...")
            
            latest_file = self.find_latest_file(pattern)
            if not latest_file:
                print(f"   ❌ No files found for {pattern}")
                continue
            
            try:
                if latest_file.suffix == '.csv':
                    df = pd.read_csv(latest_file)
                    quality_report = self.assess_data_quality(df, data_type)
                    
                    if quality_report['quality_score'] > 50:
                        loaded_data[data_type] = df
                        quality_reports[data_type] = quality_report
                        print(f"   ✅ Loaded {len(df):,} rows from {latest_file.name}")
                        print(f"   ✅ Quality score: {quality_report['quality_score']:.1f}/100")
                        
                        # Zobrazit sloupce pro debugging
                        if data_type == 'top_strategies_bets':
                            print(f"   📊 Columns: {list(df.columns[:10])}...")  # První 10 sloupců
                    else:
                        print(f"   ⚠️  Quality score too low: {quality_report['quality_score']:.1f}")
                        
                elif latest_file.suffix == '.json':
                    with open(latest_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    loaded_data[data_type] = data
                    print(f"   ✅ Loaded JSON from {latest_file.name}")
                    
            except Exception as e:
                print(f"   ❌ Error loading {latest_file.name}: {e}")
        
        # Summary
        print(f"\n📈 DATA LOADING SUMMARY")
        print("-" * 40)
        print(f"Files loaded: {len(loaded_data)}/{len(self.data_patterns)}")
        
        # Kontrola načtených dat
        if 'top_strategies_bets' in loaded_data:
            bets_df = loaded_data['top_strategies_bets']
            print(f"\n🔍 Bet data inspection:")
            print(f"   - Total rows: {len(bets_df)}")
            print(f"   - Columns: {list(bets_df.columns)}")
            if 'strategy' in bets_df.columns:
                print(f"   - Unique strategies: {bets_df['strategy'].nunique()}")
        
        return {
            'data': loaded_data,
            'quality_reports': quality_reports,
            'summary': {
                'files_loaded': len(loaded_data),
                'load_timestamp': datetime.now().isoformat()
            }
        }

# Load the data
data_manager = DataManager(PATHS['results_dir'])
DATA_PACKAGE = data_manager.load_all_data()

In [ ]:
# ===================================================================
# 🔍 DATA STRUCTURE DIAGNOSTICS
# ===================================================================

print("\n🔍 DIAGNOSTIKA STRUKTURY DAT")
print("=" * 70)

# 1. Kontrola bet dat
if 'top_strategies_bets' in DATA_PACKAGE['data']:
    bets_df = DATA_PACKAGE['data']['top_strategies_bets']
    print("\n📊 BET DATA STRUCTURE:")
    print(f"   - Rows: {len(bets_df):,}")
    print(f"   - Columns: {list(bets_df.columns)}")
    
    # Ukázka strategy identifikátorů
    if 'strategy_id' in bets_df.columns:
        print(f"\n   📌 Unique strategy_id values (first 5):")
        unique_strategies = bets_df['strategy_id'].unique()[:5]
        for strat in unique_strategies:
            count = len(bets_df[bets_df['strategy_id'] == strat])
            print(f"      - {strat}: {count} bets")
    
    # Kontrola důležitých sloupců
    required_cols = ['stake', 'net_result', 'bet_won', 'date']
    missing_cols = [col for col in required_cols if col not in bets_df.columns]
    if missing_cols:
        print(f"\n   ⚠️  Missing columns: {missing_cols}")
    
    print(f"\n   📌 Sample data:")
    print(bets_df[['strategy_id', 'date', 'stake', 'net_result', 'bet_won']].head())

# 2. Kontrola daily dat
if 'top_strategies_daily' in DATA_PACKAGE['data']:
    daily_df = DATA_PACKAGE['data']['top_strategies_daily']
    print("\n📊 DAILY DATA STRUCTURE:")
    print(f"   - Rows: {len(daily_df):,}")
    print(f"   - Columns: {list(daily_df.columns)}")
    
    # Ukázka strategy identifikátorů
    if 'strategy_id' in daily_df.columns:
        print(f"\n   📌 Unique strategy_id values (first 5):")
        unique_strategies = daily_df['strategy_id'].unique()[:5]
        for strat in unique_strategies:
            count = len(daily_df[daily_df['strategy_id'] == strat])
            print(f"      - {strat}: {count} days")
    
    # Kontrola return sloupců
    return_cols = [col for col in daily_df.columns if 'roi' in col.lower() or 'return' in col.lower()]
    if return_cols:
        print(f"\n   📌 Return columns found: {return_cols}")
    else:
        print(f"\n   ⚠️  No return columns found! Available numeric columns:")
        numeric_cols = daily_df.select_dtypes(include=[np.number]).columns.tolist()
        print(f"      {numeric_cols}")
    
    print(f"\n   📌 Sample data:")
    display_cols = ['strategy_id', 'date'] + return_cols[:3] if return_cols else ['strategy_id', 'date']
    display_cols = [col for col in display_cols if col in daily_df.columns]
    print(daily_df[display_cols].head())

# 3. Kontrola konzistence mezi datasety
print("\n🔗 CONSISTENCY CHECK:")
if 'top_strategies_bets' in DATA_PACKAGE['data'] and 'top_strategies_daily' in DATA_PACKAGE['data']:
    bets_strategies = set(DATA_PACKAGE['data']['top_strategies_bets']['strategy_id'].unique())
    daily_strategies = set(DATA_PACKAGE['data']['top_strategies_daily']['strategy_id'].unique())
    
    common_strategies = bets_strategies.intersection(daily_strategies)
    print(f"   - Strategies in bets data: {len(bets_strategies)}")
    print(f"   - Strategies in daily data: {len(daily_strategies)}")
    print(f"   - Common strategies: {len(common_strategies)}")
    
    if len(common_strategies) < len(bets_strategies):
        missing_in_daily = bets_strategies - daily_strategies
        print(f"\n   ⚠️  Strategies missing in daily data: {len(missing_in_daily)}")
        print(f"      First 5: {list(missing_in_daily)[:5]}")

print("\n" + "=" * 70)

In [ ]:
# ===================================================================
# 🎯 SECTION 3: STRATEGY SELECTION & VALIDATION
# ===================================================================

class StrategySelector:
    """Simplified strategy selection and validation"""
    
    def __init__(self, min_bets: int = 50, min_roi_threshold: float = -0.20, max_strategies: int = 20):
        self.min_bets = min_bets
        self.min_roi_threshold = min_roi_threshold
        self.max_strategies = max_strategies
    
    def calculate_max_drawdown(self, returns: pd.Series) -> float:
        """Calculate maximum drawdown from returns"""
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        return drawdown.min() if len(drawdown) > 0 else 0
    
    def select_strategies(self, data_package: Dict[str, Any]) -> Dict[str, Any]:
        """Select and validate top strategies"""
        print(f"\n🎯 STRATEGY SELECTION & VALIDATION")
        print("=" * 60)
        
        if 'top_strategies_bets' not in data_package['data']:
            print("❌ No bet data available for strategy selection")
            return {'error': 'No bet data available'}
        
        bet_data = data_package['data']['top_strategies_bets']
        
        # Calculate strategy metrics
        strategy_stats = self._calculate_strategy_metrics(bet_data)
        
        if strategy_stats.empty:
            print("❌ No strategies found in data")
            return {'error': 'No strategies found'}
        
        print(f"📊 Total strategies found: {len(strategy_stats)}")
        
        # Apply filters
        filtered = strategy_stats[
            (strategy_stats['total_bets'] >= self.min_bets) &
            (strategy_stats['roi'] >= self.min_roi_threshold)
        ].copy()
        
        print(f"📊 Strategies after filtering: {len(filtered)}")
        
        if len(filtered) == 0:
            print("❌ No strategies meet the criteria")
            return {'error': 'No strategies meet selection criteria'}
        
        # Calculate composite score for ranking
        filtered['composite_score'] = (
            filtered['roi'] * 0.4 +
            filtered['sharpe_ratio'] * 0.3 +
            filtered['win_rate'] * 0.2 +
            (1 / (filtered['max_drawdown'].abs() + 0.01)) * 0.1
        )
        
        # Select top strategies
        top_strategies = filtered.nlargest(self.max_strategies, 'composite_score')
        
        # Display results
        print(f"\n🏆 TOP {len(top_strategies)} SELECTED STRATEGIES:")
        print("-" * 50)
        
        for i, (_, strategy) in enumerate(top_strategies.iterrows(), 1):
            print(f"{i:2d}. {strategy['strategy'][:40]:<40} | "
                  f"ROI: {strategy['roi']:+6.1%} | "
                  f"Bets: {strategy['total_bets']:4.0f} | "
                  f"Win Rate: {strategy['win_rate']:5.1%}")
        
        return {
            'selected_strategies': top_strategies,
            'all_strategies': strategy_stats,
            'selection_criteria': {
                'min_bets': self.min_bets,
                'min_roi_threshold': self.min_roi_threshold,
                'max_strategies': self.max_strategies
            },
            'summary': {
                'total_found': len(strategy_stats),
                'total_selected': len(top_strategies),
                'avg_roi': top_strategies['roi'].mean(),
                'avg_win_rate': top_strategies['win_rate'].mean(),
                'total_bets': top_strategies['total_bets'].sum()
            }
        }
    
    def _calculate_strategy_metrics(self, bet_data: pd.DataFrame) -> pd.DataFrame:
        """Calculate metrics for each strategy"""
        # OPRAVA: Vytvoříme strategy identifikátor ze strategy_id
        if 'strategy_id' in bet_data.columns:
            bet_data['strategy'] = bet_data['strategy_id']
        elif 'strategy_index' in bet_data.columns:
            bet_data['strategy'] = 'Strategy_' + bet_data['strategy_index'].astype(str)
        else:
            print("❌ No strategy identifier found in data")
            return pd.DataFrame()
        
        strategy_stats = []
        
        for strategy, group in bet_data.groupby('strategy'):
            total_bets = len(group)
            
            # Win rate - OPRAVA: používáme bet_won místo result
            wins = (group['bet_won'] == True).sum() if 'bet_won' in group.columns else 0
            win_rate = wins / total_bets if total_bets > 0 else 0
            
            # Financial metrics - OPRAVA: používáme net_result místo profit
            if 'net_result' in group.columns and 'stake' in group.columns:
                total_profit = group['net_result'].sum()
                total_stake = group['stake'].sum()
                roi = total_profit / total_stake if total_stake > 0 else 0
                
                # Risk metrics
                returns = group['net_result'] / group['stake']
                max_drawdown = self.calculate_max_drawdown(returns)
                volatility = returns.std() if len(returns) > 1 else 0
                sharpe_ratio = returns.mean() / volatility if volatility > 0 else 0
            else:
                roi = max_drawdown = volatility = sharpe_ratio = 0
            
            strategy_stats.append({
                'strategy': str(strategy),  # Ensure string type
                'total_bets': total_bets,
                'win_rate': win_rate,
                'roi': roi,
                'max_drawdown': max_drawdown,
                'volatility': volatility,
                'sharpe_ratio': sharpe_ratio
            })
        
        return pd.DataFrame(strategy_stats)

# Select strategies
strategy_selector = StrategySelector()
STRATEGY_SELECTION = strategy_selector.select_strategies(DATA_PACKAGE)

In [ ]:
# ===================================================================
# 📋 PART 1 COMPLETION SUMMARY
# ===================================================================

print(f"\n" + "=" * 80)
print("📋 PART 1: PROJECT SETUP & DATA PIPELINE - COMPLETION SUMMARY")
print("=" * 80)

print(f"✅ Project paths configured: {len(PATHS)} directories")
print(f"✅ Data loaded: {len(DATA_PACKAGE['data'])}/{len(data_manager.data_patterns)} datasets")

if 'selected_strategies' in STRATEGY_SELECTION:
    print(f"✅ Strategies selected: {len(STRATEGY_SELECTION['selected_strategies'])} top performers")
    print(f"   - Average ROI: {STRATEGY_SELECTION['summary']['avg_roi']:+.1%}")
    print(f"   - Total bets analyzed: {STRATEGY_SELECTION['summary']['total_bets']:,}")

# OPRAVA: Použití správných názvů klíčů
print(f"\n📊 Available for Risk Analysis:")
data_mappings = {
    'bets': 'top_strategies_bets',
    'daily_financial': 'top_strategies_daily', 
    'monthly': 'top_strategies_monthly',
    'correlations': 'strategy_correlations'
}

for display_name, actual_key in data_mappings.items():
    status = '✅' if actual_key in DATA_PACKAGE['data'] else '❌'
    print(f"   - {display_name}: {status}")

print(f"\n💾 Session data saved in variables:")
print(f"   - PATHS: Project paths and directories")
print(f"   - DATA_PACKAGE: Loaded datasets with quality reports")
print(f"   - STRATEGY_SELECTION: Selected strategies for analysis")

print(f"\n🎯 Ready to proceed to Part 2: Core Risk Analysis")
print("=" * 80)

---

# ⚠️ PART 2: CORE RISK ANALYSIS

**Maximum drawdown, Monte Carlo VaR, and risk-adjusted performance metrics**

---

In [ ]:
# ===================================================================
# 📊 RISK ASSESSMENT ANALYSIS - PART 2: CORE RISK ANALYSIS
# ===================================================================
# 
# Core risk metrics s focus na:
# - Maximum Drawdown Analysis (peak-to-trough with duration)
# - Monte Carlo VaR Simulations (10,000 scenarios)
# - Risk-Adjusted Performance Metrics (Sharpe, Sortino, Calmar, Omega)
#
# Location: notebooks/analysis/risk_assessment.ipynb
# Part 2/4: Core Risk Analysis
# Requires: Part 1 variables (DATA_PACKAGE, STRATEGY_SELECTION, etc.)
# ===================================================================

# Verify Part 1 data availability
if 'DATA_PACKAGE' not in locals():
    print("❌ Part 1 data not available. Please run Part 1 first.")
    raise RuntimeError("Part 1 data required")

print("🔄 Starting Part 2: Core Risk Analysis")
print("=" * 60)

In [ ]:
# ===================================================================
# 📉 SECTION 1: RISK METRICS BASE CLASS
# ===================================================================

class RiskMetricsCalculator:
    """Base class for risk metrics calculations"""
    
    @staticmethod
    def calculate_drawdown_series(returns: pd.Series) -> pd.Series:
        """Calculate drawdown series from returns"""
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        return (cumulative - running_max) / running_max
    
    @staticmethod
    def calculate_max_drawdown(returns: pd.Series) -> float:
        """Calculate maximum drawdown"""
        drawdown = RiskMetricsCalculator.calculate_drawdown_series(returns)
        return drawdown.min() if len(drawdown) > 0 else 0
    
    @staticmethod
    def calculate_sharpe_ratio(returns: pd.Series, risk_free_rate: float = 0.02) -> float:
        """Calculate annualized Sharpe ratio"""
        daily_rf = risk_free_rate / 252
        excess_returns = returns - daily_rf
        if excess_returns.std() == 0:
            return 0.0
        return excess_returns.mean() / excess_returns.std() * np.sqrt(252)
    
    @staticmethod
    def calculate_sortino_ratio(returns: pd.Series, risk_free_rate: float = 0.02) -> float:
        """Calculate Sortino ratio"""
        daily_rf = risk_free_rate / 252
        excess_returns = returns - daily_rf
        downside_returns = excess_returns[excess_returns < 0]
        
        if len(downside_returns) == 0 or downside_returns.std() == 0:
            return float('inf') if excess_returns.mean() > 0 else 0.0
        
        return excess_returns.mean() / downside_returns.std() * np.sqrt(252)
    
    @staticmethod
    def calculate_calmar_ratio(returns: pd.Series, max_drawdown: float) -> float:
        """Calculate Calmar ratio"""
        if max_drawdown == 0:
            return float('inf') if returns.mean() > 0 else 0.0
        annual_return = returns.mean() * 252
        return annual_return / abs(max_drawdown)

In [ ]:
# ===================================================================
# 📉 SECTION 2: DRAWDOWN ANALYSIS
# ===================================================================

class DrawdownAnalyzer(RiskMetricsCalculator):
    """Simplified drawdown analysis"""
    
    def analyze_drawdowns(self, returns: pd.Series, strategy_name: str) -> Dict[str, Any]:
        """Analyze drawdown characteristics"""
        if len(returns) < 2:
            return {'error': 'Insufficient data'}
        
        # Calculate drawdown series
        drawdown_series = self.calculate_drawdown_series(returns)
        max_dd_idx = drawdown_series.idxmin()
        max_dd = drawdown_series.min()
        
        # Find recovery
        recovery_idx = None
        if max_dd_idx in drawdown_series.index:
            post_dd = drawdown_series.loc[max_dd_idx:]
            recovery_mask = post_dd >= -0.001  # Nearly recovered
            if recovery_mask.any():
                recovery_idx = post_dd[recovery_mask].index[0]
        
        # Calculate durations
        drawdown_start_idx = drawdown_series[:max_dd_idx][drawdown_series[:max_dd_idx] >= -0.001].index[-1] if max_dd_idx in drawdown_series.index else None
        
        drawdown_days = (max_dd_idx - drawdown_start_idx).days if drawdown_start_idx and hasattr(max_dd_idx - drawdown_start_idx, 'days') else len(drawdown_series[:max_dd_idx])
        recovery_days = (recovery_idx - max_dd_idx).days if recovery_idx and hasattr(recovery_idx - max_dd_idx, 'days') else None
        
        return {
            'strategy_name': strategy_name,
            'max_drawdown': max_dd,
            'max_drawdown_pct': max_dd * 100,
            'max_drawdown_date': max_dd_idx,
            'drawdown_duration_days': drawdown_days,
            'recovery_duration_days': recovery_days,
            'is_recovered': recovery_idx is not None,
            'time_underwater_pct': (drawdown_series < -0.01).sum() / len(drawdown_series) * 100,
            'volatility_annualized': returns.std() * np.sqrt(252),
            'drawdown_series': drawdown_series
        }
    
    def analyze_all_strategies(self, data_package: Dict[str, Any], 
                             strategy_selection: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze drawdowns for all selected strategies"""
        print(f"\n📉 MAXIMUM DRAWDOWN ANALYSIS")
        print("-" * 50)
        
        if 'top_strategies_daily' not in data_package['data']:
            print("❌ Daily financial data not available")
            return {'error': 'Daily financial data required'}
        
        # OPRAVA: Zkontrolujeme, zda máme selected_strategies
        if 'error' in strategy_selection or 'selected_strategies' not in strategy_selection:
            print("❌ No strategies selected for analysis")
            return {'error': 'No strategies selected'}
        
        daily_data = data_package['data']['top_strategies_daily']
        selected_strategies = strategy_selection['selected_strategies']
        
        # OPRAVA: Přidáme strategy sloupec do daily_data pokud chybí
        if 'strategy' not in daily_data.columns:
            if 'strategy_id' in daily_data.columns:
                daily_data['strategy'] = daily_data['strategy_id']
            elif 'strategy_index' in daily_data.columns:
                daily_data['strategy'] = 'Strategy_' + daily_data['strategy_index'].astype(str)
            else:
                print("❌ No strategy identifier in daily data")
                return {'error': 'No strategy identifier in daily data'}
        
        results = {}
        summary_stats = []
        
        for strategy_name in selected_strategies['strategy'].tolist():
            print(f"\n🔍 Analyzing: {strategy_name[:50]}")
            
            # Get strategy data
            strategy_data = daily_data[daily_data['strategy'] == strategy_name].copy()
            
            if len(strategy_data) < 10:
                print(f"   ⚠️  Insufficient data: {len(strategy_data)} days")
                continue
            
            # Get returns - OPRAVA: Hledáme správné názvy sloupců
            strategy_data = strategy_data.sort_values('date')
            
            # Zkusíme různé možné názvy sloupců pro returns
            returns = None
            for col in ['daily_roi', 'daily_return', 'roi', 'return', 'net_return']:
                if col in strategy_data.columns:
                    returns = strategy_data[col]
                    break
            
            # Pokud nenajdeme returns, zkusíme je vypočítat
            if returns is None or returns.empty:
                if 'net_result' in strategy_data.columns and 'stake' in strategy_data.columns:
                    returns = strategy_data['net_result'] / strategy_data['stake']
                else:
                    print(f"   ❌ No return data available")
                    continue
            
            # Set date index
            if 'date' in strategy_data.columns:
                returns.index = pd.to_datetime(strategy_data['date'])
            
            # Analyze drawdown
            dd_analysis = self.analyze_drawdowns(returns, strategy_name)
            
            if 'error' not in dd_analysis:
                results[strategy_name] = dd_analysis
                summary_stats.append({
                    'strategy': strategy_name,
                    'max_drawdown_pct': dd_analysis['max_drawdown_pct'],
                    'drawdown_duration_days': dd_analysis['drawdown_duration_days'],
                    'recovery_duration_days': dd_analysis['recovery_duration_days'],
                    'is_recovered': dd_analysis['is_recovered'],
                    'time_underwater_pct': dd_analysis['time_underwater_pct']
                })
                
                print(f"   📊 Max Drawdown: {dd_analysis['max_drawdown_pct']:.1f}%")
                print(f"   ⏱️  Duration: {dd_analysis['drawdown_duration_days']} days")
                print(f"   🔄 Recovery: {dd_analysis['recovery_duration_days'] or 'Ongoing'}")
        
        # Portfolio summary
        if summary_stats:
            summary_df = pd.DataFrame(summary_stats)
            portfolio_summary = {
                'total_strategies_analyzed': len(results),
                'avg_max_drawdown': summary_df['max_drawdown_pct'].mean(),
                'worst_drawdown': summary_df['max_drawdown_pct'].min(),
                'avg_recovery_time': summary_df['recovery_duration_days'].mean(),
                'recovery_rate': summary_df['is_recovered'].mean() * 100
            }
            
            print(f"\n📊 PORTFOLIO DRAWDOWN SUMMARY")
            print("-" * 40)
            print(f"Average max drawdown: {portfolio_summary['avg_max_drawdown']:.1f}%")
            print(f"Worst drawdown: {portfolio_summary['worst_drawdown']:.1f}%")
            print(f"Recovery rate: {portfolio_summary['recovery_rate']:.1f}%")
        else:
            portfolio_summary = {}
            summary_df = pd.DataFrame()
        
        return {
            'individual_results': results,
            'summary_statistics': summary_df,
            'portfolio_summary': portfolio_summary,
            'analysis_timestamp': datetime.now().isoformat()
        }

# Run drawdown analysis
drawdown_analyzer = DrawdownAnalyzer()
DRAWDOWN_RESULTS = drawdown_analyzer.analyze_all_strategies(DATA_PACKAGE, STRATEGY_SELECTION)

In [ ]:
# ===================================================================
# 🎲 SECTION 3: MONTE CARLO VaR ANALYSIS
# ===================================================================

class MonteCarloVaR(RiskMetricsCalculator):
    """Simplified Monte Carlo Value at Risk analysis"""
    
    def __init__(self, n_simulations: int = 10000):
        self.n_simulations = n_simulations
    
    def calculate_var(self, returns: pd.Series, confidence_levels: List[float] = [0.95, 0.99]) -> Dict[str, Any]:
        """Calculate VaR using historical and Monte Carlo methods"""
        if len(returns) < 30:
            return {'error': 'Insufficient data for VaR calculation'}
        
        results = {}
        
        # Historical VaR
        historical_var = {}
        for conf in confidence_levels:
            var_level = np.percentile(returns, (1 - conf) * 100)
            historical_var[f'{conf:.0%}'] = var_level
        
        results['historical_var'] = historical_var
        
        # Monte Carlo simulation (bootstrap)
        simulated_returns = np.random.choice(returns, size=(self.n_simulations, len(returns)), replace=True)
        portfolio_returns = simulated_returns.sum(axis=1)
        
        # Calculate VaR from simulations
        mc_var = {}
        mc_cvar = {}
        
        for conf in confidence_levels:
            var_level = np.percentile(portfolio_returns, (1 - conf) * 100)
            cvar_level = portfolio_returns[portfolio_returns <= var_level].mean()
            mc_var[f'{conf:.0%}'] = var_level / len(returns)  # Daily VaR
            mc_cvar[f'{conf:.0%}'] = cvar_level / len(returns)  # Daily CVaR
        
        results['monte_carlo_var'] = mc_var
        results['monte_carlo_cvar'] = mc_cvar
        
        # Distribution metrics
        results['distribution_metrics'] = {
            'mean': portfolio_returns.mean() / len(returns),
            'std': portfolio_returns.std() / len(returns),
            'skewness': stats.skew(portfolio_returns),
            'kurtosis': stats.kurtosis(portfolio_returns),
            'prob_loss': (portfolio_returns < 0).mean()
        }
        
        return results
    
    def analyze_portfolio_var(self, data_package: Dict[str, Any], 
                            strategy_selection: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze VaR for all strategies"""
        print(f"\n🎲 MONTE CARLO VaR ANALYSIS ({self.n_simulations:,} simulations)")
        print("-" * 60)
        
        if 'top_strategies_daily' not in data_package['data']:
            print("❌ Daily financial data not available")
            return {'error': 'Daily financial data required'}
        
        # OPRAVA: Zkontrolujeme, zda máme selected_strategies
        if 'error' in strategy_selection or 'selected_strategies' not in strategy_selection:
            print("❌ No strategies selected for analysis")
            return {'error': 'No strategies selected'}
        
        daily_data = data_package['data']['top_strategies_daily']
        selected_strategies = strategy_selection['selected_strategies']
        
        # OPRAVA: Přidáme strategy sloupec do daily_data pokud chybí
        if 'strategy' not in daily_data.columns:
            if 'strategy_id' in daily_data.columns:
                daily_data['strategy'] = daily_data['strategy_id']
            elif 'strategy_index' in daily_data.columns:
                daily_data['strategy'] = 'Strategy_' + daily_data['strategy_index'].astype(str)
            else:
                print("❌ No strategy identifier in daily data")
                return {'error': 'No strategy identifier in daily data'}
        
        individual_results = {}
        
        for strategy_name in selected_strategies['strategy'].tolist():
            strategy_data = daily_data[daily_data['strategy'] == strategy_name].copy()
            
            if len(strategy_data) < 30:
                continue
            
            strategy_data = strategy_data.sort_values('date')
            
            # OPRAVA: Hledáme správné názvy sloupců pro returns
            returns = None
            for col in ['daily_roi', 'daily_return', 'roi', 'return', 'net_return']:
                if col in strategy_data.columns:
                    returns = strategy_data[col]
                    break
            
            # Pokud nenajdeme returns, zkusíme je vypočítat
            if returns is None or returns.empty:
                if 'net_result' in strategy_data.columns and 'stake' in strategy_data.columns:
                    returns = strategy_data['net_result'] / strategy_data['stake']
                else:
                    continue
            
            print(f"\n🎲 Analyzing VaR for {strategy_name[:40]}...")
            
            var_results = self.calculate_var(returns)
            
            if 'error' not in var_results:
                var_results['strategy_name'] = strategy_name
                individual_results[strategy_name] = var_results
                
                print(f"   📊 VaR 95%: {var_results['monte_carlo_var']['95%']:+.2%}")
                print(f"   📊 VaR 99%: {var_results['monte_carlo_var']['99%']:+.2%}")
                print(f"   📊 Probability of Loss: {var_results['distribution_metrics']['prob_loss']:.1%}")
        
        # Summary statistics
        summary_stats = {}
        if individual_results:
            var_95_values = [r['monte_carlo_var']['95%'] for r in individual_results.values()]
            var_99_values = [r['monte_carlo_var']['99%'] for r in individual_results.values()]
            
            summary_stats = {
                'total_strategies': len(individual_results),
                'avg_var_95': np.mean(var_95_values),
                'worst_var_95': min(var_95_values),
                'avg_var_99': np.mean(var_99_values),
                'worst_var_99': min(var_99_values)
            }
            
            print(f"\n📊 VaR SUMMARY")
            print("-" * 30)
            print(f"Average VaR 95%: {summary_stats['avg_var_95']:+.2%}")
            print(f"Worst VaR 95%: {summary_stats['worst_var_95']:+.2%}")
        
        return {
            'individual_results': individual_results,
            'summary_statistics': summary_stats,
            'simulation_parameters': {
                'n_simulations': self.n_simulations,
                'confidence_levels': [0.95, 0.99],
                'method': 'Bootstrap resampling'
            },
            'analysis_timestamp': datetime.now().isoformat()
        }

# Run VaR analysis
var_analyzer = MonteCarloVaR(n_simulations=10000)
VAR_RESULTS = var_analyzer.analyze_portfolio_var(DATA_PACKAGE, STRATEGY_SELECTION)

In [ ]:
# ===================================================================
# 📈 SECTION 4: RISK-ADJUSTED PERFORMANCE METRICS
# ===================================================================

class PerformanceAnalyzer(RiskMetricsCalculator):
    """Simplified risk-adjusted performance analysis"""
    
    def __init__(self, risk_free_rate: float = 0.02):
        self.risk_free_rate = risk_free_rate
    
    def calculate_omega_ratio(self, returns: pd.Series, threshold: float = 0.0) -> float:
        """Calculate Omega ratio"""
        excess_returns = returns - threshold
        gains = excess_returns[excess_returns > 0].sum()
        losses = abs(excess_returns[excess_returns <= 0].sum())
        
        if losses == 0:
            return float('inf') if gains > 0 else 0.0
        return gains / losses
    
    def analyze_performance(self, returns: pd.Series, max_drawdown: float) -> Dict[str, float]:
        """Calculate all performance metrics"""
        return {
            'annual_return': returns.mean() * 252,
            'annual_volatility': returns.std() * np.sqrt(252),
            'sharpe_ratio': self.calculate_sharpe_ratio(returns, self.risk_free_rate),
            'sortino_ratio': self.calculate_sortino_ratio(returns, self.risk_free_rate),
            'calmar_ratio': self.calculate_calmar_ratio(returns, max_drawdown),
            'omega_ratio': self.calculate_omega_ratio(returns),
            'max_drawdown': max_drawdown,
            'total_return': (1 + returns).prod() - 1,
            'positive_days_pct': (returns > 0).mean(),
            'skewness': stats.skew(returns),
            'kurtosis': stats.kurtosis(returns)
        }
    
    def analyze_all_strategies(self, data_package: Dict[str, Any],
                             strategy_selection: Dict[str, Any],
                             drawdown_results: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze performance for all strategies"""
        print(f"\n📈 RISK-ADJUSTED PERFORMANCE ANALYSIS")
        print("-" * 50)
        
        if 'top_strategies_daily' not in data_package['data']:
            print("❌ Daily financial data not available")
            return {'error': 'Daily financial data required'}
        
        # OPRAVA: Zkontrolujeme, zda máme selected_strategies
        if 'error' in strategy_selection or 'selected_strategies' not in strategy_selection:
            print("❌ No strategies selected for analysis")
            return {'error': 'No strategies selected'}
        
        daily_data = data_package['data']['top_strategies_daily']
        selected_strategies = strategy_selection['selected_strategies']
        
        # OPRAVA: Přidáme strategy sloupec do daily_data pokud chybí
        if 'strategy' not in daily_data.columns:
            if 'strategy_id' in daily_data.columns:
                daily_data['strategy'] = daily_data['strategy_id']
            elif 'strategy_index' in daily_data.columns:
                daily_data['strategy'] = 'Strategy_' + daily_data['strategy_index'].astype(str)
            else:
                print("❌ No strategy identifier in daily data")
                return {'error': 'No strategy identifier in daily data'}
        
        performance_results = {}
        performance_summary = []
        
        for strategy_name in selected_strategies['strategy'].tolist():
            print(f"\n📊 Analyzing performance for: {strategy_name[:50]}")
            
            # Get strategy data
            strategy_data = daily_data[daily_data['strategy'] == strategy_name].copy()
            
            if len(strategy_data) < 10:
                continue
            
            strategy_data = strategy_data.sort_values('date')
            
            # OPRAVA: Hledáme správné názvy sloupců pro returns
            returns = None
            for col in ['daily_roi', 'daily_return', 'roi', 'return', 'net_return']:
                if col in strategy_data.columns:
                    returns = strategy_data[col]
                    break
            
            # Pokud nenajdeme returns, zkusíme je vypočítat
            if returns is None or returns.empty:
                if 'net_result' in strategy_data.columns and 'stake' in strategy_data.columns:
                    returns = strategy_data['net_result'] / strategy_data['stake']
                else:
                    continue
            
            # Get max drawdown from previous analysis
            max_drawdown = 0
            if 'individual_results' in drawdown_results and strategy_name in drawdown_results['individual_results']:
                max_drawdown = abs(drawdown_results['individual_results'][strategy_name]['max_drawdown'])
            else:
                max_drawdown = abs(self.calculate_max_drawdown(returns))
            
            # Calculate performance metrics
            metrics = self.analyze_performance(returns, max_drawdown)
            metrics['strategy_name'] = strategy_name
            metrics['total_days'] = len(returns)
            
            performance_results[strategy_name] = metrics
            performance_summary.append(metrics)
            
            # Display key metrics
            print(f"   📊 Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
            print(f"   📊 Annual Return: {metrics['annual_return']:+.1%}")
            print(f"   📊 Annual Volatility: {metrics['annual_volatility']:.1%}")
        
        # Portfolio summary
        portfolio_summary = {}
        summary_df = pd.DataFrame()
        
        if performance_summary:
            summary_df = pd.DataFrame(performance_summary)
            
            portfolio_summary = {
                'total_strategies': len(performance_summary),
                'avg_sharpe_ratio': summary_df['sharpe_ratio'].mean(),
                'avg_sortino_ratio': summary_df['sortino_ratio'].mean(),
                'avg_annual_return': summary_df['annual_return'].mean(),
                'avg_annual_volatility': summary_df['annual_volatility'].mean(),
                'best_sharpe_strategy': summary_df.loc[summary_df['sharpe_ratio'].idxmax(), 'strategy_name'] if len(summary_df) > 0 else None,
                'best_return_strategy': summary_df.loc[summary_df['annual_return'].idxmax(), 'strategy_name'] if len(summary_df) > 0 else None
            }
            
            print(f"\n📊 PERFORMANCE SUMMARY")
            print("-" * 40)
            print(f"Average Sharpe Ratio: {portfolio_summary['avg_sharpe_ratio']:.2f}")
            print(f"Average Annual Return: {portfolio_summary['avg_annual_return']:+.1%}")
        
        return {
            'individual_results': performance_results,
            'summary_statistics': summary_df,
            'portfolio_summary': portfolio_summary,
            'analysis_parameters': {
                'risk_free_rate': self.risk_free_rate,
                'analysis_date': datetime.now().isoformat()
            }
        }

# Run performance analysis
performance_analyzer = PerformanceAnalyzer(risk_free_rate=0.02)
PERFORMANCE_RESULTS = performance_analyzer.analyze_all_strategies(
    DATA_PACKAGE, STRATEGY_SELECTION, DRAWDOWN_RESULTS
)

In [ ]:
# ===================================================================
# 📋 PART 2 COMPLETION SUMMARY
# ===================================================================

print(f"\n" + "=" * 80)
print("📋 PART 2: CORE RISK ANALYSIS - COMPLETION SUMMARY")
print("=" * 80)

# Drawdown Analysis Summary
if 'individual_results' in DRAWDOWN_RESULTS:
    dd_count = len(DRAWDOWN_RESULTS['individual_results'])
    avg_dd = DRAWDOWN_RESULTS.get('portfolio_summary', {}).get('avg_max_drawdown', 0)
    recovery_rate = DRAWDOWN_RESULTS.get('portfolio_summary', {}).get('recovery_rate', 0)
    
    print(f"✅ Maximum Drawdown Analysis:")
    print(f"   - Strategies analyzed: {dd_count}")
    print(f"   - Average max drawdown: {avg_dd:.1f}%")
    print(f"   - Recovery rate: {recovery_rate:.1f}%")

# VaR Analysis Summary  
if 'individual_results' in VAR_RESULTS:
    var_count = len(VAR_RESULTS['individual_results'])
    avg_var = VAR_RESULTS.get('summary_statistics', {}).get('avg_var_95', 0)
    n_sims = VAR_RESULTS.get('simulation_parameters', {}).get('n_simulations', 0)
    
    print(f"✅ Monte Carlo VaR Analysis:")
    print(f"   - Strategies analyzed: {var_count}")
    print(f"   - Simulations performed: {n_sims:,}")
    print(f"   - Average VaR 95%: {avg_var:+.1%}")

# Performance Analysis Summary
if 'individual_results' in PERFORMANCE_RESULTS:
    perf_count = len(PERFORMANCE_RESULTS['individual_results'])
    avg_sharpe = PERFORMANCE_RESULTS.get('portfolio_summary', {}).get('avg_sharpe_ratio', 0)
    avg_return = PERFORMANCE_RESULTS.get('portfolio_summary', {}).get('avg_annual_return', 0)
    
    print(f"✅ Risk-Adjusted Performance Analysis:")
    print(f"   - Strategies analyzed: {perf_count}")
    print(f"   - Average Sharpe ratio: {avg_sharpe:.2f}")
    print(f"   - Average annual return: {avg_return:+.1%}")

print(f"\n💾 Session data available:")
print(f"   - DRAWDOWN_RESULTS: Maximum drawdown analysis")
print(f"   - VAR_RESULTS: Monte Carlo VaR simulations")
print(f"   - PERFORMANCE_RESULTS: Risk-adjusted performance metrics")

print(f"\n🎯 Ready to proceed to Part 3: Portfolio & Recovery Analysis")
print("=" * 80)

---

# 🔗 PART 3: PORTFOLIO & RECOVERY ANALYSIS

**Correlation analysis, diversification insights, and recovery patterns**

---

In [ ]:
# ===================================================================
# 📊 RISK ASSESSMENT ANALYSIS - PART 3: PORTFOLIO & RECOVERY ANALYSIS
# ===================================================================
# 
# Portfolio diversification a recovery resilience analysis:
# - Portfolio Correlation Analysis (strategy diversification mapping)  
# - Recovery Period Analysis (drawdown recovery patterns)
# - Correlation-based portfolio optimization
# - Recovery resilience scoring
#
# Location: notebooks/analysis/risk_assessment.ipynb
# Part 3/4: Portfolio & Recovery Analysis
# Requires: Parts 1-2 variables (DATA_PACKAGE, DRAWDOWN_RESULTS, etc.)
# ===================================================================

# Verify previous parts data availability
required_vars = ['DATA_PACKAGE', 'STRATEGY_SELECTION', 'DRAWDOWN_RESULTS', 'VAR_RESULTS', 'PERFORMANCE_RESULTS']
missing_vars = []
available_vars = []

print("🔍 Checking required variables from previous parts...")
print("-" * 60)

for var in required_vars:
    if var in globals():
        available_vars.append(var)
        print(f"✅ {var} is available")
    else:
        missing_vars.append(var)
        print(f"❌ {var} is MISSING")

if missing_vars:
    print(f"\n❌ Missing data from previous parts: {missing_vars}")
    print("\n📋 INSTRUKCE PRO OPRAVU:")
    print("1. Ujistěte se, že jste spustili VŠECHNY buňky v PART 1")
    print("2. Ujistěte se, že jste spustili VŠECHNY buňky v PART 2")
    print("3. Pokud jste restartovali kernel, musíte spustit části 1 a 2 znovu")
    print("\nTIP: Použijte 'Run All Above' nebo 'Run All' pro spuštění všech předchozích buněk")
    
    # Poskytněte informaci o tom, co může být načteno z dat
    if 'DATA_PACKAGE' in globals():
        print(f"\n📊 DATA_PACKAGE je dostupný s klíči: {list(DATA_PACKAGE.get('data', {}).keys())}")
    
    raise RuntimeError("Previous parts data required - please run Parts 1-2 first")

print(f"\n✅ All required variables are available!")
print("🔄 Starting Part 3: Portfolio & Recovery Analysis")
print("=" * 70)

In [ ]:
# ===================================================================
# 🔗 SECTION 1: CORRELATION ANALYSIS
# ===================================================================

class CorrelationAnalyzer:
    """Simplified portfolio correlation analysis"""
    
    def calculate_correlations(self, daily_data: pd.DataFrame, strategy_names: List[str]) -> pd.DataFrame:
        """Calculate correlation matrix for strategies"""
        returns_dict = {}
        
        # OPRAVA: Přidáme strategy sloupec pokud chybí
        if 'strategy' not in daily_data.columns:
            if 'strategy_id' in daily_data.columns:
                daily_data['strategy'] = daily_data['strategy_id']
            elif 'strategy_index' in daily_data.columns:
                daily_data['strategy'] = 'Strategy_' + daily_data['strategy_index'].astype(str)
        
        for strategy in strategy_names:
            strategy_data = daily_data[daily_data['strategy'] == strategy].copy()
            
            if len(strategy_data) < 30:
                continue
            
            strategy_data = strategy_data.sort_values('date')
            
            # OPRAVA: Hledáme správné názvy sloupců pro returns
            returns = None
            for col in ['daily_roi', 'daily_return', 'roi', 'return', 'net_return']:
                if col in strategy_data.columns:
                    returns = strategy_data[col]
                    break
            
            # Pokud nenajdeme returns, zkusíme je vypočítat
            if returns is None or returns.empty:
                if 'net_result' in strategy_data.columns and 'stake' in strategy_data.columns:
                    returns = strategy_data['net_result'] / strategy_data['stake']
                else:
                    continue
            
            if not returns.empty:
                returns_dict[strategy[:30]] = returns.values  # Truncate name for display
        
        if len(returns_dict) < 2:
            return pd.DataFrame()
        
        # Align lengths
        min_length = min(len(r) for r in returns_dict.values())
        aligned_returns = {k: v[-min_length:] for k, v in returns_dict.items()}
        
        # Create DataFrame and calculate correlations
        returns_df = pd.DataFrame(aligned_returns)
        return returns_df.corr()
    
    def analyze_diversification(self, correlation_matrix: pd.DataFrame) -> Dict[str, Any]:
        """Analyze diversification metrics"""
        if correlation_matrix.empty:
            return {'error': 'Empty correlation matrix'}
        
        n = len(correlation_matrix)
        
        # Extract upper triangle (excluding diagonal)
        upper_triangle = np.triu(correlation_matrix.values, k=1)
        correlations = upper_triangle[upper_triangle != 0]
        
        # Diversification ratio (simplified)
        equal_weights = np.ones(n) / n
        portfolio_variance = np.dot(equal_weights, np.dot(correlation_matrix.values, equal_weights))
        portfolio_vol = np.sqrt(portfolio_variance)
        avg_vol = 1.0  # Assuming unit volatilities
        diversification_ratio = avg_vol / portfolio_vol if portfolio_vol > 0 else 1.0
        
        # High correlation pairs
        high_corr_pairs = []
        for i in range(n):
            for j in range(i+1, n):
                corr = correlation_matrix.iloc[i, j]
                if abs(corr) > 0.7:
                    high_corr_pairs.append({
                        'strategy_1': correlation_matrix.index[i],
                        'strategy_2': correlation_matrix.index[j],
                        'correlation': corr
                    })
        
        return {
            'mean_correlation': np.mean(correlations) if len(correlations) > 0 else 0,
            'std_correlation': np.std(correlations) if len(correlations) > 0 else 0,
            'diversification_ratio': diversification_ratio,
            'high_correlation_pairs': high_corr_pairs,
            'effective_strategies': len(correlation_matrix),
            'concentration_score': 1 - (len(correlation_matrix) / n) if n > 0 else 0
        }
    
    def calculate_optimal_weights(self, correlation_matrix: pd.DataFrame) -> Dict[str, Any]:
        """Calculate minimum variance portfolio weights"""
        if correlation_matrix.empty:
            return {'error': 'Empty correlation matrix'}
        
        n = len(correlation_matrix)
        
        try:
            # Minimum variance portfolio (simplified)
            inv_cov = np.linalg.inv(correlation_matrix.values)
            ones = np.ones((n, 1))
            
            weights = np.dot(inv_cov, ones) / np.dot(ones.T, np.dot(inv_cov, ones))
            weights = weights.flatten()
            
            # Ensure non-negative weights
            weights = np.maximum(weights, 0)
            weights = weights / np.sum(weights)
            
            # Calculate risk reduction vs equal weights
            equal_weights = np.ones(n) / n
            equal_var = np.dot(equal_weights, np.dot(correlation_matrix.values, equal_weights))
            opt_var = np.dot(weights, np.dot(correlation_matrix.values, weights))
            risk_reduction = (equal_var - opt_var) / equal_var if equal_var > 0 else 0
            
            return {
                'optimal_weights': dict(zip(correlation_matrix.index, weights)),
                'equal_weights': dict(zip(correlation_matrix.index, equal_weights)),
                'risk_reduction_benefit': risk_reduction,
                'max_weight': np.max(weights),
                'herfindahl_index': np.sum(weights**2)
            }
            
        except np.linalg.LinAlgError:
            equal_weights = np.ones(n) / n
            return {
                'optimal_weights': dict(zip(correlation_matrix.index, equal_weights)),
                'equal_weights': dict(zip(correlation_matrix.index, equal_weights)),
                'error': 'Matrix inversion failed'
            }
    
    def analyze_portfolio_correlations(self, data_package: Dict[str, Any],
                                     strategy_selection: Dict[str, Any]) -> Dict[str, Any]:
        """Complete correlation analysis"""
        print(f"\n🔗 PORTFOLIO CORRELATION ANALYSIS")
        print("-" * 60)
        
        if 'top_strategies_daily' not in data_package['data']:
            print("❌ Daily financial data not available")
            return {'error': 'Daily financial data required'}
        
        # OPRAVA: Zkontrolujeme, zda máme selected_strategies
        if 'error' in strategy_selection or 'selected_strategies' not in strategy_selection:
            print("❌ No strategies selected for analysis")
            return {'error': 'No strategies selected'}
        
        daily_data = data_package['data']['top_strategies_daily']
        strategy_names = strategy_selection['selected_strategies']['strategy'].tolist()
        
        # Calculate correlations
        print(f"🔗 Calculating correlations for {len(strategy_names)} strategies...")
        correlation_matrix = self.calculate_correlations(daily_data, strategy_names)
        
        if correlation_matrix.empty:
            return {'error': 'Failed to calculate correlations'}
        
        print(f"   ✅ Correlation matrix: {correlation_matrix.shape[0]}x{correlation_matrix.shape[1]}")
        
        # Analyze diversification
        correlation_analysis = self.analyze_diversification(correlation_matrix)
        print(f"   📊 Mean correlation: {correlation_analysis['mean_correlation']:.3f}")
        print(f"   📊 Diversification ratio: {correlation_analysis['diversification_ratio']:.2f}")
        
        # Calculate optimal weights
        optimal_weights = self.calculate_optimal_weights(correlation_matrix)
        if 'risk_reduction_benefit' in optimal_weights:
            print(f"   📊 Risk reduction potential: {optimal_weights['risk_reduction_benefit']:.1%}")
        
        # Generate recommendations
        recommendations = self._generate_recommendations(correlation_analysis, optimal_weights)
        
        return {
            'correlation_matrix': correlation_matrix,
            'correlation_analysis': correlation_analysis,
            'optimal_weights': optimal_weights,
            'diversification_recommendations': recommendations,
            'analysis_timestamp': datetime.now().isoformat()
        }
    
    def _generate_recommendations(self, correlation_analysis: Dict[str, Any],
                                optimal_weights: Dict[str, Any]) -> List[Dict[str, str]]:
        """Generate diversification recommendations"""
        recommendations = []
        
        # Diversification ratio
        div_ratio = correlation_analysis.get('diversification_ratio', 1.0)
        if div_ratio < 1.2:
            recommendations.append({
                'type': 'warning',
                'title': 'Limited Diversification',
                'description': f'Diversification ratio of {div_ratio:.2f} suggests limited risk reduction',
                'action': 'Consider adding uncorrelated strategies'
            })
        elif div_ratio > 1.5:
            recommendations.append({
                'type': 'positive',
                'title': 'Good Diversification',
                'description': f'Diversification ratio of {div_ratio:.2f} indicates effective risk reduction',
                'action': 'Maintain current diversification approach'
            })
        
        # High correlations
        high_corr_count = len(correlation_analysis.get('high_correlation_pairs', []))
        if high_corr_count > 0:
            recommendations.append({
                'type': 'warning',
                'title': 'High Correlations Detected',
                'description': f'Found {high_corr_count} strategy pairs with correlation > 0.7',
                'action': 'Review and potentially reduce exposure to correlated strategies'
            })
        
        # Concentration risk
        max_weight = optimal_weights.get('max_weight', 0)
        if max_weight > 0.4:
            recommendations.append({
                'type': 'warning',
                'title': 'Concentration Risk',
                'description': f'Optimal allocation suggests {max_weight:.1%} in single strategy',
                'action': 'Consider position limits to prevent over-concentration'
            })
        
        return recommendations

# Run correlation analysis
correlation_analyzer = CorrelationAnalyzer()
CORRELATION_RESULTS = correlation_analyzer.analyze_portfolio_correlations(DATA_PACKAGE, STRATEGY_SELECTION)

In [ ]:
# ===================================================================
# 📋 SECTION 2: RECOVERY ANALYSIS  
# ===================================================================

class RecoveryAnalyzer:
    """Simplified recovery period analysis"""
    
    def analyze_recovery_metrics(self, drawdown_results: Dict[str, Any]) -> Dict[str, Any]:
        """Extract and analyze recovery metrics from drawdown results"""
        print(f"\n📋 RECOVERY PERIOD ANALYSIS")
        print("-" * 50)
        
        if 'individual_results' not in drawdown_results:
            print("❌ Drawdown results not available")
            return {'error': 'Drawdown results required'}
        
        recovery_summary = []
        
        for strategy_name, dd_results in drawdown_results['individual_results'].items():
            recovery_time = dd_results.get('recovery_duration_days')
            
            recovery_metrics = {
                'strategy': strategy_name,
                'max_drawdown_pct': dd_results.get('max_drawdown_pct', 0),
                'drawdown_days': dd_results.get('drawdown_duration_days', 0),
                'recovery_days': recovery_time if recovery_time is not None else 999,  # Large number for ongoing
                'is_recovered': dd_results.get('is_recovered', False),
                'time_underwater_pct': dd_results.get('time_underwater_pct', 0),
                'total_recovery_time': (dd_results.get('drawdown_duration_days', 0) + 
                                      (recovery_time if recovery_time is not None else 0))
            }
            
            # Calculate recovery efficiency
            if recovery_time is not None and recovery_time > 0:
                recovery_metrics['recovery_efficiency'] = abs(dd_results.get('max_drawdown_pct', 1)) / recovery_time
            else:
                recovery_metrics['recovery_efficiency'] = 0
            
            # Calculate resilience score (simplified)
            resilience_score = self._calculate_resilience_score(recovery_metrics)
            recovery_metrics['resilience_score'] = resilience_score
            
            recovery_summary.append(recovery_metrics)
        
        # Convert to DataFrame for analysis
        recovery_df = pd.DataFrame(recovery_summary)
        
        # Portfolio summary
        portfolio_recovery = {
            'total_strategies': len(recovery_summary),
            'avg_recovery_time': recovery_df[recovery_df['is_recovered']]['recovery_days'].mean() if any(recovery_df['is_recovered']) else None,
            'median_recovery_time': recovery_df[recovery_df['is_recovered']]['recovery_days'].median() if any(recovery_df['is_recovered']) else None,
            'recovery_success_rate': recovery_df['is_recovered'].mean(),
            'avg_time_underwater': recovery_df['time_underwater_pct'].mean(),
            'avg_resilience_score': recovery_df['resilience_score'].mean(),
            'fastest_recovery_strategy': recovery_df.loc[recovery_df['recovery_days'].idxmin(), 'strategy'] if len(recovery_df) > 0 else None,
            'most_resilient_strategy': recovery_df.loc[recovery_df['resilience_score'].idxmax(), 'strategy'] if len(recovery_df) > 0 else None
        }
        
        # Display summary
        print(f"\n📊 RECOVERY SUMMARY")
        print("-" * 40)
        print(f"Strategies analyzed: {portfolio_recovery['total_strategies']}")
        if portfolio_recovery['avg_recovery_time'] is not None:
            print(f"Average recovery time: {portfolio_recovery['avg_recovery_time']:.1f} days")
        print(f"Recovery success rate: {portfolio_recovery['recovery_success_rate']:.1%}")
        print(f"Average resilience score: {portfolio_recovery['avg_resilience_score']:.2f}")
        
        return {
            'recovery_summary': recovery_df,
            'portfolio_recovery': portfolio_recovery,
            'analysis_timestamp': datetime.now().isoformat()
        }
    
    def _calculate_resilience_score(self, metrics: Dict[str, Any]) -> float:
        """Calculate resilience score (0-1)"""
        score = 0.0
        
        # Recovery speed component (30%)
        recovery_days = metrics.get('recovery_days', 999)
        if recovery_days < 30:
            score += 0.3
        elif recovery_days < 60:
            score += 0.2
        elif recovery_days < 90:
            score += 0.1
        
        # Recovery success component (30%)
        if metrics.get('is_recovered', False):
            score += 0.3
        
        # Time underwater component (20%)
        time_underwater = metrics.get('time_underwater_pct', 100)
        if time_underwater < 20:
            score += 0.2
        elif time_underwater < 40:
            score += 0.1
        
        # Drawdown magnitude component (20%)
        max_dd = abs(metrics.get('max_drawdown_pct', 100))
        if max_dd < 10:
            score += 0.2
        elif max_dd < 20:
            score += 0.1
        
        return min(1.0, score)

# Run recovery analysis
recovery_analyzer = RecoveryAnalyzer()
RECOVERY_RESULTS = recovery_analyzer.analyze_recovery_metrics(DRAWDOWN_RESULTS)

In [ ]:
# ===================================================================
# 📊 SECTION 3: INTEGRATED PORTFOLIO INSIGHTS
# ===================================================================

class PortfolioInsightsGenerator:
    """Generate integrated portfolio insights from all analyses"""
    
    def generate_insights(self, all_results: Dict[str, Any]) -> Dict[str, Any]:
        """Generate comprehensive portfolio insights"""
        print(f"\n📊 INTEGRATED PORTFOLIO INSIGHTS")
        print("-" * 50)
        
        # Extract key metrics
        insights = self._extract_portfolio_metrics(all_results)
        
        # Calculate portfolio score
        portfolio_score = self._calculate_portfolio_score(insights)
        insights['portfolio_score'] = portfolio_score
        
        # Generate strategic recommendations
        insights['strategic_recommendations'] = self._generate_strategic_recommendations(
            insights, portfolio_score
        )
        
        # Identify optimization opportunities
        insights['optimization_opportunities'] = self._identify_opportunities(insights)
        
        # Display summary
        print(f"\n📊 PORTFOLIO ASSESSMENT")
        print("-" * 40)
        print(f"Portfolio Score: {portfolio_score:.1f}/100")
        print(f"Risk Level: {self._get_risk_level(insights)}")
        print(f"Diversification: {self._get_diversification_rating(insights)}")
        print(f"Recovery Capability: {self._get_recovery_rating(insights)}")
        print(f"Strategic Recommendations: {len(insights['strategic_recommendations'])}")
        
        return insights
    
    def _extract_portfolio_metrics(self, all_results: Dict[str, Any]) -> Dict[str, Any]:
        """Extract key metrics from all analyses"""
        metrics = {
            'portfolio_characteristics': {},
            'risk_profile': {},
            'performance_metrics': {}
        }
        
        # Correlation metrics
        if 'correlation_results' in all_results and 'correlation_analysis' in all_results['correlation_results']:
            corr = all_results['correlation_results']['correlation_analysis']
            metrics['portfolio_characteristics']['diversification_ratio'] = corr.get('diversification_ratio', 1.0)
            metrics['portfolio_characteristics']['mean_correlation'] = corr.get('mean_correlation', 0)
        
        # Drawdown metrics
        if 'drawdown_results' in all_results and 'portfolio_summary' in all_results['drawdown_results']:
            dd = all_results['drawdown_results']['portfolio_summary']
            metrics['risk_profile']['avg_max_drawdown'] = dd.get('avg_max_drawdown', 0)
            metrics['risk_profile']['worst_drawdown'] = dd.get('worst_drawdown', 0)
        
        # VaR metrics
        if 'var_results' in all_results and 'summary_statistics' in all_results['var_results']:
            var = all_results['var_results']['summary_statistics']
            metrics['risk_profile']['avg_var_95'] = var.get('avg_var_95', 0)
            metrics['risk_profile']['worst_var_95'] = var.get('worst_var_95', 0)
        
        # Performance metrics
        if 'performance_results' in all_results and 'portfolio_summary' in all_results['performance_results']:
            perf = all_results['performance_results']['portfolio_summary']
            metrics['performance_metrics']['avg_sharpe_ratio'] = perf.get('avg_sharpe_ratio', 0)
            metrics['performance_metrics']['avg_annual_return'] = perf.get('avg_annual_return', 0)
        
        # Recovery metrics
        if 'recovery_results' in all_results and 'portfolio_recovery' in all_results['recovery_results']:
            rec = all_results['recovery_results']['portfolio_recovery']
            metrics['risk_profile']['avg_recovery_time'] = rec.get('avg_recovery_time', 0)
            metrics['risk_profile']['recovery_success_rate'] = rec.get('recovery_success_rate', 0)
        
        return metrics
    
    def _calculate_portfolio_score(self, metrics: Dict[str, Any]) -> float:
        """Calculate overall portfolio quality score (0-100)"""
        score = 0.0
        
        # Diversification (25 points)
        div_ratio = metrics['portfolio_characteristics'].get('diversification_ratio', 1.0)
        score += min(25, (div_ratio - 1.0) * 50)
        
        # Performance (25 points)
        sharpe = metrics['performance_metrics'].get('avg_sharpe_ratio', 0)
        score += min(25, max(0, sharpe * 12.5))
        
        # Risk control (25 points)
        max_dd = abs(metrics['risk_profile'].get('worst_drawdown', 50))
        score += min(25, max(0, 25 - max_dd * 0.5))
        
        # Recovery (25 points)
        recovery_rate = metrics['risk_profile'].get('recovery_success_rate', 0)
        score += recovery_rate * 25
        
        return max(0, min(100, score))
    
    def _generate_strategic_recommendations(self, metrics: Dict[str, Any], 
                                          portfolio_score: float) -> List[Dict[str, str]]:
        """Generate strategic recommendations"""
        recommendations = []
        
        # Portfolio quality recommendation
        if portfolio_score < 60:
            recommendations.append({
                'category': 'Portfolio Optimization',
                'recommendation': 'Comprehensive portfolio restructuring needed',
                'priority': 'High',
                'rationale': f'Portfolio score of {portfolio_score:.1f}/100 below target'
            })
        
        # Diversification recommendation
        div_ratio = metrics['portfolio_characteristics'].get('diversification_ratio', 1.0)
        if div_ratio < 1.3:
            recommendations.append({
                'category': 'Diversification',
                'recommendation': 'Increase portfolio diversification',
                'priority': 'Medium',
                'rationale': f'Diversification ratio of {div_ratio:.2f} is suboptimal'
            })
        
        # Risk management recommendation
        max_dd = abs(metrics['risk_profile'].get('worst_drawdown', 0))
        if max_dd > 25:
            recommendations.append({
                'category': 'Risk Management',
                'recommendation': 'Implement enhanced risk controls',
                'priority': 'High',
                'rationale': f'Maximum drawdown of {max_dd:.1f}% exceeds risk limits'
            })
        
        # Performance recommendation
        sharpe = metrics['performance_metrics'].get('avg_sharpe_ratio', 0)
        if sharpe < 1.0:
            recommendations.append({
                'category': 'Performance',
                'recommendation': 'Focus on improving risk-adjusted returns',
                'priority': 'Medium',
                'rationale': f'Sharpe ratio of {sharpe:.2f} below target of 1.0'
            })
        
        return recommendations
    
    def _identify_opportunities(self, metrics: Dict[str, Any]) -> List[Dict[str, str]]:
        """Identify optimization opportunities"""
        opportunities = []
        
        # Check each metric area
        div_ratio = metrics['portfolio_characteristics'].get('diversification_ratio', 1.0)
        if div_ratio < 1.5:
            opportunities.append({
                'area': 'Diversification',
                'opportunity': 'Add uncorrelated strategies',
                'potential_improvement': '15-25% risk reduction'
            })
        
        recovery_time = metrics['risk_profile'].get('avg_recovery_time', 0)
        if recovery_time and recovery_time > 45:
            opportunities.append({
                'area': 'Recovery',
                'opportunity': 'Improve recovery characteristics',
                'potential_improvement': 'Faster drawdown recovery'
            })
        
        return opportunities
    
    def _get_risk_level(self, metrics: Dict[str, Any]) -> str:
        """Determine overall risk level"""
        max_dd = abs(metrics['risk_profile'].get('worst_drawdown', 0))
        var_95 = abs(metrics['risk_profile'].get('avg_var_95', 0))
        
        if max_dd < 15 and var_95 < 0.05:
            return "Low Risk"
        elif max_dd < 25 and var_95 < 0.10:
            return "Medium Risk"
        else:
            return "High Risk"
    
    def _get_diversification_rating(self, metrics: Dict[str, Any]) -> str:
        """Rate diversification quality"""
        div_ratio = metrics['portfolio_characteristics'].get('diversification_ratio', 1.0)
        
        if div_ratio >= 1.5:
            return "Excellent"
        elif div_ratio >= 1.3:
            return "Good"
        elif div_ratio >= 1.1:
            return "Fair"
        else:
            return "Poor"
    
    def _get_recovery_rating(self, metrics: Dict[str, Any]) -> str:
        """Rate recovery capability"""
        recovery_rate = metrics['risk_profile'].get('recovery_success_rate', 0)
        
        if recovery_rate >= 0.8:
            return "Strong"
        elif recovery_rate >= 0.6:
            return "Moderate"
        else:
            return "Weak"

# Generate integrated insights
insights_generator = PortfolioInsightsGenerator()

ALL_RESULTS = {
    'drawdown_results': DRAWDOWN_RESULTS,
    'var_results': VAR_RESULTS,
    'performance_results': PERFORMANCE_RESULTS,
    'correlation_results': CORRELATION_RESULTS,
    'recovery_results': RECOVERY_RESULTS
}

INTEGRATED_INSIGHTS = insights_generator.generate_insights(ALL_RESULTS)

In [ ]:
# ===================================================================
# 📋 PART 3 COMPLETION SUMMARY
# ===================================================================

print(f"\n" + "=" * 80)
print("📋 PART 3: PORTFOLIO & RECOVERY ANALYSIS - COMPLETION SUMMARY")
print("=" * 80)

# Correlation Analysis Summary
if 'correlation_analysis' in CORRELATION_RESULTS:
    corr_analysis = CORRELATION_RESULTS['correlation_analysis']
    print(f"✅ Portfolio Correlation Analysis:")
    print(f"   - Diversification ratio: {corr_analysis.get('diversification_ratio', 0):.2f}")
    print(f"   - Mean correlation: {corr_analysis.get('mean_correlation', 0):+.3f}")
    
    if 'optimal_weights' in CORRELATION_RESULTS:
        risk_reduction = CORRELATION_RESULTS['optimal_weights'].get('risk_reduction_benefit', 0)
        print(f"   - Risk reduction potential: {risk_reduction:.1%}")

# Recovery Analysis Summary
if 'portfolio_recovery' in RECOVERY_RESULTS:
    portfolio_recovery = RECOVERY_RESULTS['portfolio_recovery']
    print(f"✅ Recovery Period Analysis:")
    if portfolio_recovery.get('avg_recovery_time') is not None:
        print(f"   - Average recovery time: {portfolio_recovery['avg_recovery_time']:.1f} days")
    print(f"   - Recovery success rate: {portfolio_recovery.get('recovery_success_rate', 0):.1%}")
    print(f"   - Average resilience score: {portfolio_recovery.get('avg_resilience_score', 0):.2f}")

# Integrated Insights Summary
if 'portfolio_score' in INTEGRATED_INSIGHTS:
    portfolio_score = INTEGRATED_INSIGHTS['portfolio_score']
    strategic_recs = len(INTEGRATED_INSIGHTS.get('strategic_recommendations', []))
    optimization_opps = len(INTEGRATED_INSIGHTS.get('optimization_opportunities', []))
    
    print(f"✅ Integrated Portfolio Analysis:")
    print(f"   - Portfolio quality score: {portfolio_score:.1f}/100")
    print(f"   - Strategic recommendations: {strategic_recs}")
    print(f"   - Optimization opportunities: {optimization_opps}")

print(f"\n💾 Session data available:")
print(f"   - CORRELATION_RESULTS: Strategy correlation and diversification")
print(f"   - RECOVERY_RESULTS: Recovery period analysis")
print(f"   - INTEGRATED_INSIGHTS: Combined portfolio insights")

print(f"\n🎯 Ready to proceed to Part 4: Summary & Export")
print("=" * 80)

---

# 📤 PART 4: SUMMARY & EXPORT

**Executive summary, professional visualizations, and comprehensive reporting**

---

In [ ]:
# ===================================================================
# 📊 RISK ASSESSMENT ANALYSIS - PART 4: SUMMARY & EXPORT
# ===================================================================
# 
# Final comprehensive summary and export:
# - Executive Risk Assessment Summary
# - Actionable Implementation Recommendations  
# - Professional Visualization Export
# - Comprehensive Risk Report Generation
# - Strategic Decision Framework
#
# Location: notebooks/analysis/risk_assessment.ipynb
# Part 4/4: Summary & Export
# Requires: Parts 1-3 variables (all previous analysis results)
# ===================================================================

# Verify all previous parts data availability
required_vars = [
    'DATA_PACKAGE', 'STRATEGY_SELECTION', 'DRAWDOWN_RESULTS', 
    'VAR_RESULTS', 'PERFORMANCE_RESULTS', 'CORRELATION_RESULTS', 
    'RECOVERY_RESULTS', 'INTEGRATED_INSIGHTS'
]
missing_vars = []
available_vars = []

print("🔍 Checking required variables from previous parts...")
print("-" * 70)

for var in required_vars:
    if var in globals():
        available_vars.append(var)
        print(f"✅ {var} is available")
    else:
        missing_vars.append(var)
        print(f"❌ {var} is MISSING")

if missing_vars:
    print(f"\n❌ Missing data from previous parts: {missing_vars}")
    print("\n📋 INSTRUKCE PRO OPRAVU:")
    print("1. Ujistěte se, že jste spustili VŠECHNY buňky v PART 1")
    print("2. Ujistěte se, že jste spustili VŠECHNY buňky v PART 2")
    print("3. Ujistěte se, že jste spustili VŠECHNY buňky v PART 3")
    print("4. Pokud jste restartovali kernel, musíte spustit části 1-3 znovu")
    print("\nTIP: Použijte 'Run All Above' nebo 'Run All' pro spuštění všech předchozích buněk")
    
    # Poskytněte informaci o tom, co může být načteno z dat
    if 'DATA_PACKAGE' in globals():
        print(f"\n📊 DATA_PACKAGE je dostupný s klíči: {list(DATA_PACKAGE.get('data', {}).keys())}")
    
    raise RuntimeError("Previous parts data required - please run Parts 1-3 first")

print(f"\n✅ All {len(available_vars)} required variables are available!")
print("🔄 Starting Part 4: Summary & Export")
print("=" * 70)

In [ ]:
# ===================================================================
# 📋 SECTION 1: EXECUTIVE SUMMARY GENERATOR
# ===================================================================

class ExecutiveSummaryGenerator:
    """Generate executive-level summary and recommendations"""
    
    def generate_summary(self, all_data: Dict[str, Any]) -> Dict[str, Any]:
        """Generate comprehensive executive summary"""
        print(f"\n📋 GENERATING EXECUTIVE RISK ASSESSMENT SUMMARY")
        print("-" * 60)
        
        # Extract key information
        portfolio_overview = self._create_portfolio_overview(all_data)
        risk_assessment = self._assess_risk_profile(all_data)
        strategic_assessment = self._create_strategic_assessment(all_data)
        bottom_line = self._generate_bottom_line(risk_assessment, strategic_assessment)
        
        # Compile executive summary
        executive_summary = {
            'analysis_metadata': {
                'analysis_date': datetime.now().isoformat(),
                'analysis_scope': 'Comprehensive Hockey Betting Strategy Risk Assessment',
                'confidence_level': '95% statistical confidence'
            },
            'portfolio_overview': portfolio_overview,
            'risk_assessment': risk_assessment,
            'strategic_assessment': strategic_assessment,
            'bottom_line_assessment': bottom_line,
            'executive_recommendations': self._generate_executive_recommendations(
                risk_assessment, strategic_assessment
            )
        }
        
        # Display summary
        self._display_executive_summary(executive_summary)
        
        return executive_summary
    
    def _create_portfolio_overview(self, all_data: Dict[str, Any]) -> Dict[str, Any]:
        """Create portfolio overview section"""
        strategy_selection = all_data.get('strategy_selection', {})
        data_package = all_data.get('data_package', {})
        
        return {
            'total_strategies_analyzed': len(strategy_selection.get('selected_strategies', [])),
            'total_historical_bets': strategy_selection.get('summary', {}).get('total_bets', 0),
            'portfolio_avg_roi': strategy_selection.get('summary', {}).get('avg_roi', 0),
            'data_quality_score': np.mean([
                report.get('quality_score', 75) 
                for report in data_package.get('quality_reports', {}).values()
            ]) if data_package.get('quality_reports') else 75.0
        }
    
    def _assess_risk_profile(self, all_data: Dict[str, Any]) -> Dict[str, Any]:
        """Assess overall risk profile"""
        # Extract risk metrics
        drawdown_summary = all_data.get('drawdown_results', {}).get('portfolio_summary', {})
        var_summary = all_data.get('var_results', {}).get('summary_statistics', {})
        
        # Calculate risk score
        max_dd = abs(drawdown_summary.get('worst_drawdown', 0))
        avg_var = abs(var_summary.get('avg_var_95', 0))
        
        # Determine risk level
        if max_dd < 15 and avg_var < 0.05:
            risk_level = 'Low Risk'
            risk_score = 1.0
        elif max_dd < 25 and avg_var < 0.10:
            risk_level = 'Medium Risk'
            risk_score = 2.0
        else:
            risk_level = 'High Risk'
            risk_score = 3.0
        
        return {
            'overall_risk_level': risk_level,
            'risk_score': risk_score,
            'max_drawdown': max_dd,
            'avg_var_95': avg_var * 100,
            'risk_factors': {
                'drawdown_risk': 'High' if max_dd > 25 else 'Medium' if max_dd > 15 else 'Low',
                'var_risk': 'High' if avg_var > 0.10 else 'Medium' if avg_var > 0.05 else 'Low'
            }
        }
    
    def _create_strategic_assessment(self, all_data: Dict[str, Any]) -> Dict[str, Any]:
        """Create strategic assessment"""
        integrated_insights = all_data.get('integrated_insights', {})
        portfolio_score = integrated_insights.get('portfolio_score', 50)
        
        # Determine quality rating
        if portfolio_score >= 80:
            quality_rating = 'Excellent'
        elif portfolio_score >= 65:
            quality_rating = 'Good'
        elif portfolio_score >= 50:
            quality_rating = 'Fair'
        else:
            quality_rating = 'Poor'
        
        return {
            'portfolio_quality_rating': quality_rating,
            'portfolio_score': portfolio_score,
            'key_strengths': self._identify_strengths(all_data),
            'key_concerns': self._identify_concerns(all_data)
        }
    
    def _identify_strengths(self, all_data: Dict[str, Any]) -> List[str]:
        """Identify portfolio strengths"""
        strengths = []
        
        # Check diversification
        div_ratio = all_data.get('correlation_results', {}).get('correlation_analysis', {}).get('diversification_ratio', 1.0)
        if div_ratio > 1.4:
            strengths.append(f"Strong diversification (ratio: {div_ratio:.2f})")
        
        # Check performance
        avg_sharpe = all_data.get('performance_results', {}).get('portfolio_summary', {}).get('avg_sharpe_ratio', 0)
        if avg_sharpe > 1.2:
            strengths.append(f"Excellent risk-adjusted returns (Sharpe: {avg_sharpe:.2f})")
        
        # Check recovery
        recovery_rate = all_data.get('recovery_results', {}).get('portfolio_recovery', {}).get('recovery_success_rate', 0)
        if recovery_rate > 0.8:
            strengths.append(f"Strong recovery capability ({recovery_rate:.1%} success rate)")
        
        if not strengths:
            strengths.append("Portfolio meets basic risk management standards")
        
        return strengths
    
    def _identify_concerns(self, all_data: Dict[str, Any]) -> List[str]:
        """Identify portfolio concerns"""
        concerns = []
        
        # Check drawdown
        max_dd = abs(all_data.get('drawdown_results', {}).get('portfolio_summary', {}).get('worst_drawdown', 0))
        if max_dd > 25:
            concerns.append(f"High maximum drawdown risk ({max_dd:.1f}%)")
        
        # Check diversification
        div_ratio = all_data.get('correlation_results', {}).get('correlation_analysis', {}).get('diversification_ratio', 1.0)
        if div_ratio < 1.2:
            concerns.append(f"Limited diversification benefits (ratio: {div_ratio:.2f})")
        
        # Check VaR
        worst_var = abs(all_data.get('var_results', {}).get('summary_statistics', {}).get('worst_var_95', 0))
        if worst_var > 0.15:
            concerns.append(f"High worst-case loss potential (VaR 95%: {worst_var:.1%})")
        
        return concerns
    
    def _generate_bottom_line(self, risk_assessment: Dict[str, Any], 
                            strategic_assessment: Dict[str, Any]) -> Dict[str, str]:
        """Generate bottom-line go/no-go assessment"""
        portfolio_score = strategic_assessment['portfolio_score']
        risk_level = risk_assessment['overall_risk_level']
        
        if portfolio_score >= 75 and risk_level != 'High Risk':
            decision = 'GO'
            rationale = 'Portfolio meets all key performance and risk criteria'
            action = 'Proceed with current strategy allocation'
        elif portfolio_score >= 60:
            decision = 'CONDITIONAL GO'
            rationale = 'Portfolio shows promise but requires optimization'
            action = 'Implement recommended improvements before scaling'
        else:
            decision = 'NO GO'
            rationale = 'Portfolio does not meet minimum standards'
            action = 'Substantial restructuring required'
        
        return {
            'go_no_go_decision': decision,
            'decision_rationale': rationale,
            'key_action_required': action
        }
    
    def _generate_executive_recommendations(self, risk_assessment: Dict[str, Any],
                                          strategic_assessment: Dict[str, Any]) -> List[Dict[str, str]]:
        """Generate executive-level recommendations"""
        recommendations = []
        
        # Risk management
        if risk_assessment['risk_score'] >= 2.5:
            recommendations.append({
                'category': 'Risk Management',
                'priority': 'High',
                'recommendation': 'Implement enhanced risk controls',
                'timeline': 'Immediate',
                'expected_outcome': 'Reduced portfolio volatility and drawdown risk'
            })
        
        # Portfolio optimization
        if strategic_assessment['portfolio_score'] < 70:
            recommendations.append({
                'category': 'Portfolio Optimization',
                'priority': 'High',
                'recommendation': 'Restructure portfolio for improved performance',
                'timeline': '1-2 months',
                'expected_outcome': 'Enhanced risk-adjusted returns'
            })
        
        return recommendations
    
    def _display_executive_summary(self, summary: Dict[str, Any]):
        """Display formatted executive summary"""
        print(f"\n📊 EXECUTIVE RISK ASSESSMENT SUMMARY")
        print("=" * 60)
        
        # Portfolio Overview
        overview = summary['portfolio_overview']
        print(f"\nPortfolio Overview:")
        print(f"  • Strategies Analyzed: {overview['total_strategies_analyzed']}")
        print(f"  • Historical Bets: {overview['total_historical_bets']:,}")
        print(f"  • Average ROI: {overview['portfolio_avg_roi']:+.1%}")
        
        # Risk Assessment
        risk = summary['risk_assessment']
        print(f"\nRisk Assessment:")
        print(f"  • Overall Risk Level: {risk['overall_risk_level']} (Score: {risk['risk_score']:.1f}/3.0)")
        print(f"  • Max Drawdown: {risk['max_drawdown']:.1f}%")
        print(f"  • VaR 95%: {risk['avg_var_95']:.1f}%")
        
        # Bottom Line
        bottom_line = summary['bottom_line_assessment']
        print(f"\nBottom Line Assessment:")
        print(f"  • Decision: {bottom_line['go_no_go_decision']}")
        print(f"  • Rationale: {bottom_line['decision_rationale']}")
        print(f"  • Key Action: {bottom_line['key_action_required']}")

# Generate executive summary
summary_generator = ExecutiveSummaryGenerator()

ALL_DATA = {
    'data_package': DATA_PACKAGE,
    'strategy_selection': STRATEGY_SELECTION,
    'drawdown_results': DRAWDOWN_RESULTS,
    'var_results': VAR_RESULTS,
    'performance_results': PERFORMANCE_RESULTS,
    'correlation_results': CORRELATION_RESULTS,
    'recovery_results': RECOVERY_RESULTS,
    'integrated_insights': INTEGRATED_INSIGHTS
}

EXECUTIVE_SUMMARY = summary_generator.generate_summary(ALL_DATA)

In [ ]:
# ===================================================================
# 📈 SECTION 2: VISUALIZATION CREATOR
# ===================================================================

class RiskVisualizationCreator:
    """Create professional risk assessment visualizations"""
    
    def __init__(self, export_dir: Path):
        self.export_dir = export_dir
        self.export_dir.mkdir(parents=True, exist_ok=True)
    
    def create_all_visualizations(self, all_data: Dict[str, Any]) -> List[str]:
        """Create all risk assessment visualizations"""
        print(f"\n📈 CREATING RISK VISUALIZATIONS")
        print("-" * 50)
        
        created_files = []
        
        if not PLOTLY_AVAILABLE:
            print("⚠️ Plotly not available, skipping interactive visualizations")
            return created_files
        
        # Create each visualization
        visualizations = [
            ('drawdown_analysis', self._create_drawdown_chart),
            ('var_analysis', self._create_var_chart),
            ('risk_return_scatter', self._create_risk_return_chart),
            ('correlation_heatmap', self._create_correlation_heatmap),
            ('portfolio_dashboard', self._create_portfolio_dashboard)
        ]
        
        for viz_name, viz_func in visualizations:
            try:
                filename = viz_func(all_data)
                if filename:
                    created_files.append(filename)
                    print(f"   ✅ Created: {filename}")
            except Exception as e:
                print(f"   ❌ Error creating {viz_name}: {e}")
        
        print(f"\n✅ Created {len(created_files)} visualizations")
        return created_files
    
    def _create_drawdown_chart(self, all_data: Dict[str, Any]) -> Optional[str]:
        """Create drawdown analysis chart"""
        drawdown_results = all_data.get('drawdown_results', {})
        if 'summary_statistics' not in drawdown_results:
            return None
        
        summary_df = drawdown_results['summary_statistics']
        
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=['Maximum Drawdown by Strategy', 'Recovery Time Analysis']
        )
        
        # Drawdown bar chart
        fig.add_trace(
            go.Bar(
                x=summary_df['strategy'].str[:15],
                y=summary_df['max_drawdown_pct'],
                name='Max Drawdown (%)',
                marker_color='rgba(255, 99, 132, 0.8)'
            ),
            row=1, col=1
        )
        
        # Recovery time bar chart
        fig.add_trace(
            go.Bar(
                x=summary_df['strategy'].str[:15],
                y=summary_df['recovery_duration_days'],
                name='Recovery Time (days)',
                marker_color='rgba(54, 162, 235, 0.8)'
            ),
            row=1, col=2
        )
        
        fig.update_layout(
            height=500,
            title_text="Drawdown Risk Analysis",
            showlegend=False
        )
        
        filename = "risk_assessment_drawdown_analysis.html"
        fig.write_html(str(self.export_dir / filename))
        return filename
    
    def _create_var_chart(self, all_data: Dict[str, Any]) -> Optional[str]:
        """Create VaR analysis chart"""
        var_results = all_data.get('var_results', {})
        if 'individual_results' not in var_results:
            return None
        
        # Extract VaR data
        strategies = []
        var_95 = []
        var_99 = []
        
        for strategy, results in var_results['individual_results'].items():
            strategies.append(strategy[:15])
            var_95.append(abs(results['monte_carlo_var']['95%']) * 100)
            var_99.append(abs(results['monte_carlo_var']['99%']) * 100)
        
        fig = go.Figure()
        
        fig.add_trace(go.Bar(name='VaR 95%', x=strategies, y=var_95,
                           marker_color='rgba(255, 206, 86, 0.8)'))
        fig.add_trace(go.Bar(name='VaR 99%', x=strategies, y=var_99,
                           marker_color='rgba(255, 99, 132, 0.8)'))
        
        fig.update_layout(
            title='Value at Risk Analysis',
            xaxis_title='Strategy',
            yaxis_title='VaR (%)',
            barmode='group',
            height=500
        )
        
        filename = "risk_assessment_var_analysis.html"
        fig.write_html(str(self.export_dir / filename))
        return filename
    
    def _create_risk_return_chart(self, all_data: Dict[str, Any]) -> Optional[str]:
        """Create risk-return scatter plot"""
        perf_results = all_data.get('performance_results', {})
        if 'summary_statistics' not in perf_results:
            return None
        
        summary_df = perf_results['summary_statistics']
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=summary_df['annual_volatility'] * 100,
            y=summary_df['annual_return'] * 100,
            mode='markers+text',
            text=summary_df['strategy_name'].str[:10],
            textposition='top center',
            marker=dict(
                size=summary_df['sharpe_ratio'] * 10,
                color=summary_df['sharpe_ratio'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Sharpe Ratio")
            )
        ))
        
        fig.update_layout(
            title='Risk-Return Profile',
            xaxis_title='Annual Volatility (%)',
            yaxis_title='Annual Return (%)',
            height=600
        )
        
        # Add reference lines
        fig.add_hline(y=0, line_dash="dot", line_color="gray")
        
        filename = "risk_assessment_risk_return.html"
        fig.write_html(str(self.export_dir / filename))
        return filename
    
    def _create_correlation_heatmap(self, all_data: Dict[str, Any]) -> Optional[str]:
        """Create correlation heatmap"""
        corr_results = all_data.get('correlation_results', {})
        if 'correlation_matrix' not in corr_results:
            return None
        
        corr_matrix = corr_results['correlation_matrix']
        
        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale='RdBu',
            zmid=0,
            text=corr_matrix.round(2).values,
            texttemplate="%{text}",
            textfont={"size": 10}
        ))
        
        fig.update_layout(
            title='Strategy Correlation Matrix',
            height=600
        )
        
        filename = "risk_assessment_correlations.html"
        fig.write_html(str(self.export_dir / filename))
        return filename
    
    def _create_portfolio_dashboard(self, all_data: Dict[str, Any]) -> Optional[str]:
        """Create comprehensive portfolio dashboard"""
        insights = all_data.get('integrated_insights', {})
        executive_summary = all_data.get('executive_summary', {})
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=['Portfolio Score', 'Risk Metrics', 
                          'Performance Metrics', 'Key Recommendations'],
            specs=[[{"type": "indicator"}, {"type": "bar"}],
                   [{"type": "scatter"}, {"type": "table"}]]
        )
        
        # Portfolio score gauge
        portfolio_score = insights.get('portfolio_score', 50)
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=portfolio_score,
            title={'text': "Portfolio Quality Score"},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "green" if portfolio_score > 70 else "orange" if portfolio_score > 50 else "red"},
                'steps': [
                    {'range': [0, 50], 'color': "lightgray"},
                    {'range': [50, 70], 'color': "yellow"},
                    {'range': [70, 100], 'color': "lightgreen"}
                ]
            }
        ), row=1, col=1)
        
        # Risk metrics bar chart
        risk_data = executive_summary.get('risk_assessment', {})
        fig.add_trace(go.Bar(
            x=['Max Drawdown', 'VaR 95%'],
            y=[risk_data.get('max_drawdown', 0), risk_data.get('avg_var_95', 0)],
            marker_color=['red', 'orange']
        ), row=1, col=2)
        
        # Update layout
        fig.update_layout(
            height=800,
            title_text="Risk Assessment Dashboard",
            showlegend=False
        )
        
        filename = "risk_assessment_dashboard.html"
        fig.write_html(str(self.export_dir / filename))
        return filename

# Create visualizations
viz_creator = RiskVisualizationCreator(PATHS['charts_export_dir'])
VISUALIZATION_EXPORTS = viz_creator.create_all_visualizations(ALL_DATA)

In [ ]:
# ===================================================================
# 📤 SECTION 3: REPORT EXPORTER
# ===================================================================

class ReportExporter:
    """Export comprehensive risk assessment reports"""
    
    def __init__(self, export_dir: Path):
        self.export_dir = export_dir
        self.export_dir.mkdir(parents=True, exist_ok=True)
    
    def export_all_reports(self, executive_summary: Dict[str, Any], 
                         all_results: Dict[str, Any]) -> Dict[str, str]:
        """Export all report formats"""
        print(f"\n📤 EXPORTING RISK ASSESSMENT REPORTS")
        print("-" * 60)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        exported_files = {}
        
        # Export executive summary (JSON)
        exec_file = self._export_executive_json(executive_summary, timestamp)
        if exec_file:
            exported_files['executive_summary'] = exec_file
        
        # Export strategy metrics (CSV)
        metrics_file = self._export_strategy_metrics_csv(all_results, timestamp)
        if metrics_file:
            exported_files['strategy_metrics'] = metrics_file
        
        # Export HTML report
        html_file = self._export_html_report(executive_summary, timestamp)
        if html_file:
            exported_files['html_report'] = html_file
        
        print(f"\n✅ Exported {len(exported_files)} reports")
        return exported_files
    
    def _export_executive_json(self, executive_summary: Dict[str, Any], 
                             timestamp: str) -> Optional[str]:
        """Export executive summary as JSON"""
        try:
            filename = f"risk_assessment_executive_{timestamp}.json"
            filepath = self.export_dir / filename
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(executive_summary, f, indent=2, default=str)
            
            print(f"   ✅ Executive summary JSON: {filename}")
            return filename
        except Exception as e:
            print(f"   ❌ Error exporting JSON: {e}")
            return None
    
    def _export_strategy_metrics_csv(self, all_results: Dict[str, Any], 
                                   timestamp: str) -> Optional[str]:
        """Export strategy metrics as CSV"""
        try:
            # Combine metrics from different analyses
            perf_df = all_results.get('performance_results', {}).get('summary_statistics', pd.DataFrame())
            
            if perf_df.empty:
                return None
            
            filename = f"risk_assessment_metrics_{timestamp}.csv"
            filepath = self.export_dir / filename
            
            perf_df.to_csv(filepath, index=False, encoding='utf-8')
            
            print(f"   ✅ Strategy metrics CSV: {filename}")
            return filename
        except Exception as e:
            print(f"   ❌ Error exporting CSV: {e}")
            return None
    
    def _export_html_report(self, executive_summary: Dict[str, Any], 
                          timestamp: str) -> Optional[str]:
        """Export HTML executive report"""
        try:
            filename = f"risk_assessment_report_{timestamp}.html"
            filepath = self.export_dir / filename
            
            html_content = self._generate_html_content(executive_summary)
            
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(html_content)
            
            print(f"   ✅ HTML report: {filename}")
            return filename
        except Exception as e:
            print(f"   ❌ Error exporting HTML: {e}")
            return None
    
    def _generate_html_content(self, executive_summary: Dict[str, Any]) -> str:
        """Generate HTML report content"""
        overview = executive_summary.get('portfolio_overview', {})
        risk = executive_summary.get('risk_assessment', {})
        bottom_line = executive_summary.get('bottom_line_assessment', {})
        
        decision_class = bottom_line.get('go_no_go_decision', '').lower().replace(' ', '-')
        
        html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Risk Assessment Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; }}
        .header {{ background: #2c3e50; color: white; padding: 20px; text-align: center; }}
        .section {{ margin: 20px 0; padding: 20px; background: #f8f9fa; border-radius: 8px; }}
        .metric {{ display: inline-block; margin: 10px; padding: 10px; background: white; border-radius: 5px; }}
        .go {{ background: #d4edda; color: #155724; padding: 20px; border-radius: 8px; }}
        .conditional-go {{ background: #fff3cd; color: #856404; padding: 20px; border-radius: 8px; }}
        .no-go {{ background: #f8d7da; color: #721c24; padding: 20px; border-radius: 8px; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>Hockey Betting Strategy Risk Assessment</h1>
        <p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>
    </div>
    
    <div class="section">
        <h2>Portfolio Overview</h2>
        <div class="metric">Strategies: {overview.get('total_strategies_analyzed', 0)}</div>
        <div class="metric">Total Bets: {overview.get('total_historical_bets', 0):,}</div>
        <div class="metric">Avg ROI: {overview.get('portfolio_avg_roi', 0):+.1%}</div>
    </div>
    
    <div class="section">
        <h2>Risk Assessment</h2>
        <p>Risk Level: {risk.get('overall_risk_level', 'Unknown')}</p>
        <p>Max Drawdown: {risk.get('max_drawdown', 0):.1f}%</p>
        <p>VaR 95%: {risk.get('avg_var_95', 0):.1f}%</p>
    </div>
    
    <div class="{decision_class}">
        <h2>Decision: {bottom_line.get('go_no_go_decision', 'Unknown')}</h2>
        <p>{bottom_line.get('decision_rationale', '')}</p>
        <p><strong>Action Required:</strong> {bottom_line.get('key_action_required', '')}</p>
    </div>
</body>
</html>
        """
        return html

# Export reports
report_exporter = ReportExporter(PATHS['results_dir'])
REPORT_EXPORTS = report_exporter.export_all_reports(EXECUTIVE_SUMMARY, ALL_RESULTS)

In [ ]:
# ===================================================================
# 📋 FINAL COMPLETION SUMMARY
# ===================================================================

print(f"\n" + "=" * 80)
print("🎉 RISK ASSESSMENT ANALYSIS - COMPLETE!")
print("=" * 80)

print(f"\n📊 ANALYSIS SUMMARY:")
print(f"✅ Part 1: Data Pipeline - {len(DATA_PACKAGE['data'])} datasets loaded")
print(f"✅ Part 2: Core Risk Analysis - {len(DRAWDOWN_RESULTS.get('individual_results', {}))} strategies analyzed")
print(f"✅ Part 3: Portfolio Analysis - Diversification ratio: {CORRELATION_RESULTS.get('correlation_analysis', {}).get('diversification_ratio', 0):.2f}")
print(f"✅ Part 4: Summary & Export - {len(VISUALIZATION_EXPORTS)} visualizations, {len(REPORT_EXPORTS)} reports")

# Key Results
print(f"\n🎯 KEY RESULTS:")
if 'portfolio_score' in INTEGRATED_INSIGHTS:
    print(f"   - Portfolio Score: {INTEGRATED_INSIGHTS['portfolio_score']:.1f}/100")

if 'bottom_line_assessment' in EXECUTIVE_SUMMARY:
    bottom_line = EXECUTIVE_SUMMARY['bottom_line_assessment']
    print(f"   - Go/No-Go Decision: {bottom_line.get('go_no_go_decision', 'Unknown')}")

if 'risk_assessment' in EXECUTIVE_SUMMARY:
    risk = EXECUTIVE_SUMMARY['risk_assessment']
    print(f"   - Risk Level: {risk.get('overall_risk_level', 'Unknown')}")
    print(f"   - Max Drawdown: {risk.get('max_drawdown', 0):.1f}%")

print(f"\n📁 EXPORTED FILES:")
print(f"📊 Visualizations ({len(VISUALIZATION_EXPORTS)} files):")
for viz_file in VISUALIZATION_EXPORTS:
    print(f"   - {viz_file}")

print(f"\n📋 Reports ({len(REPORT_EXPORTS)} files):")
for report_type, report_file in REPORT_EXPORTS.items():
    print(f"   - {report_type}: {report_file}")

print(f"\n🎯 RECOMMENDED NEXT STEPS:")
print(f"1. Review executive summary and go/no-go decision")
print(f"2. Examine visualization dashboards for insights")
print(f"3. Implement high-priority recommendations")
print(f"4. Schedule regular risk assessment updates")

print(f"\n💾 SESSION DATA AVAILABLE:")
print(f"   - EXECUTIVE_SUMMARY: Complete executive insights")
print(f"   - ALL_DATA: Combined analysis results")
print(f"   - VISUALIZATION_EXPORTS: Created chart files")
print(f"   - REPORT_EXPORTS: Exported report files")

print(f"\n🎉 Risk Assessment Complete!")
print("=" * 80)